In [28]:
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point
from folium import GeoJson

In [10]:
gdf = gpd.read_file("../resources/gadm_level2.gpkg")
print(gdf.tail())

/opt/anaconda3/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: This version of GeoPackage user_version=0x000028A0 (10400, v1.4.0) on '../resources/gadm_level2.gpkg' may only be partially supported
  return ogr_read(


           GID_2        NAME_2    GID_1              NAME_1 GID_0    NAME_0  \
48128  ZWE.9.5_2  Gwanda Urban  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48129  ZWE.9.6_2        Insiza  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48130  ZWE.9.7_2        Mangwe  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48131  ZWE.9.8_2        Matobo  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48132  ZWE.9.9_2      Plumtree  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   

                                                geometry  
48128  MULTIPOLYGON (((29.00187 -20.95896, 29.00164 -...  
48129  MULTIPOLYGON (((29.35744 -21.02481, 29.35821 -...  
48130  MULTIPOLYGON (((27.81436 -21.19368, 27.81452 -...  
48131  MULTIPOLYGON (((28.45053 -21.64326, 28.44812 -...  
48132  MULTIPOLYGON (((27.83201 -20.57665, 27.82436 -...  


In [11]:
df_images = pd.read_csv("../resources/mp16_combined.csv");
print(df_images.head());

                 IMG_ID        AUTHOR        LAT         LON  S3_Label  \
0  92_17_5276763594.jpg  42441750@N03  38.685568 -109.532951       1.0   
1  0d_ce_6392770405.jpg  68149505@N00  34.933793  103.692741       0.0   
2  2a_88_5268406683.jpg  84867026@N00  39.983433  -75.243301       0.0   
3  82_be_2515710583.jpg  75292316@N00  39.306094  -84.379291       1.0   
4  03_05_9498368699.jpg  61068860@N00   9.186625  123.581597       1.0   

   S16_Label  S365_Label   Prob_indoor  Prob_natural  Prob_urban  \
0        7.0       289.0  1.739840e-04      0.897409    0.102417   
1        1.0       122.0  9.968868e-01      0.000578    0.002535   
2        0.0       128.0  7.201538e-01      0.034871    0.244975   
3        6.0       145.0  9.050690e-05      0.516982    0.482927   
4        8.0        36.0  9.902391e-07      0.999983    0.000016   

  neighbourhood          city               county         state  \
0           NaN           NaN         Grand County          Utah   
1         

In [ ]:
geometry = [Point(xy) for xy in zip(df_images['LON'], df_images['LAT'])]
gdf_images = gpd.GeoDataFrame(df_images, geometry=geometry, crs="EPSG:4326")

joined = gpd.sjoin(gdf_images, gdf, how='left', predicate='within')

image_counts = joined.groupby(joined.index_right).size()

gdf['image_count'] = gdf.index.map(image_counts).fillna(0).astype(int)

print(gdf[['GID_2', 'NAME_2', 'image_count']].head(20))

   GID_2 NAME_2  image_count
0                        464
1                          0
2                          0
3                          2
4                        523
5                         26
6                        129
7                        729
8                          0
9                         19
10                        22
11                        67
12                      1109
13                        39
14                         1
15                         0
16                        90
17                       287
18                      2590
19                        52


In [ ]:
# Filter subdivisions with less than 100 samples
small_subdivisions = gdf[gdf['image_count'] < 100].copy()

# For each small subdivision, find neighboring subdivisions in the same country
# and merge them together
merged_gdf = gdf.copy()

for idx, small_subdiv in small_subdivisions.iterrows():
    country = small_subdiv['GID_0']
    
    # Find neighbors (subdivisions that touch this one)
    neighbors = merged_gdf[
        (merged_gdf.geometry.touches(small_subdiv.geometry)) & 
        (merged_gdf['GID_0'] == country) &
        (merged_gdf.index != idx)
    ]
    
    if len(neighbors) > 0:
        # Merge this subdivision with its neighbors using unary_union
        from shapely.ops import unary_union
        geoms_to_merge = [small_subdiv.geometry] + neighbors.geometry.tolist()
        merged_geom = unary_union(geoms_to_merge)
        merged_image_count = small_subdiv['image_count'] + neighbors['image_count'].sum()
        
        # Update the first neighbor with merged geometry and count
        first_neighbor_idx = neighbors.index[0]
        merged_gdf.loc[first_neighbor_idx, 'geometry'] = merged_geom
        merged_gdf.loc[first_neighbor_idx, 'image_count'] = merged_image_count
        
        # Drop the small subdivision (it's been merged)
        merged_gdf = merged_gdf.drop(idx)

print(merged_gdf[['GID_2', 'NAME_2', 'image_count', 'GID_0']].head(20))
print(f"Original subdivisions: {len(gdf)}, After merging: {len(merged_gdf)}")

   GID_2 NAME_2  image_count GID_0
0                        464   ABW
1                          0   ATA
2                          0   BVT
3                          2   CCK
4                        523   CUW
5                         26   CXR
6                        129   FLK
7                        729   GIB
8                          0   HMD
9                         19   IOT
10                        22   KIR
11                        67   MAF
12                      1109   MDV
13                        39   NFK
14                         1   NIU
15                         0   PCN
16                        90   SGS
17                       287   SXM
18                      2590   VAT
19                        52   XCA
Original subdivisions: 48133, After merging: 8931


In [ ]:
m = folium.Map(location=[20, 0], zoom_start=2, tiles="CartoDB positron")
print("created map")
GeoJson(merged_gdf, tooltip="NAME_2").add_to(m)
print("loaded map")
m

created map
loaded map


In [ ]:
merged_gdf.to_file("../resources/gadm_merged.gpkg", driver="GPKG")